In [ ]:
%reset -f

import time
import numpy as np
import scipy.io as spi
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.models import Model
from tensorflow.keras.layers import Dense, SimpleRNN
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler

dt = 0.01
min_t = 0
max_t = 2

t = np.arange(min_t, max_t, dt)
lt = t.shape[0]

data = spi.loadmat('BW_data.mat')
f = data['force']
y = data['y1']

f = f[0:150,:]
y = y[0:150,:][..., np.newaxis]

X_data_flatten = np.reshape(f, [f.shape[0]*f.shape[1], 1])
scaler_X = MinMaxScaler(feature_range=(-1, 1))
scaler_X.fit(X_data_flatten)
X_data_flatten_map = scaler_X.transform(X_data_flatten)
X_train = np.reshape(X_data_flatten_map, [f.shape[0], f.shape[1], 1])

y_data_flatten = np.reshape(y, [y.shape[0]*y.shape[1], y.shape[2]])
scaler_y = MinMaxScaler(feature_range=(-1, 1))
scaler_y.fit(y_data_flatten)
y_data_flatten_map = scaler_y.transform(y_data_flatten)
y_train = np.reshape(y_data_flatten_map, [y.shape[0], y.shape[1], y.shape[2]])

batch_size = 10

adam = Adam(learning_rate=0.001, decay=0.0001)
model = Sequential()
model.add(SimpleRNN(50, return_sequences=True, stateful=False, input_shape=(None, 1)))
model.add(Dense(1))
model.summary()

model.compile(loss='mean_squared_error', optimizer=adam)

for i in range(0, 50):
    model.fit(X_train, y_train, epochs=1)
    model.fit(X_train, y_train, epochs=19, verbose = 0)

# plt.plot(y_data[0,:,:], 'r')
# plt.plot( scaler_y.inverse_transform(y_data_map[0,:,:]), 'b--')

In [ ]:
# model.save_weights('./model/RNN_wts')

In [ ]:
%reset -f

import time
import numpy as np
import scipy.io as spi
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.models import Model
from tensorflow.keras.layers import Dense, SimpleRNN
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler

dt = 0.01
min_t = 0
max_t = 2

t = np.arange(min_t, max_t, dt)
lt = t.shape[0]

samples = 10000

data = spi.loadmat('BW_data.mat')
f_traintoscale = data['force'][0:150,:]
y_traintoscale = data['y1'][0:150,:]

y_traintoscale = y_traintoscale[..., np.newaxis]
print(f_traintoscale.shape, y_traintoscale.shape)


from sklearn.preprocessing import MinMaxScaler

X_data_flatten = np.reshape(f_traintoscale, [f_traintoscale.shape[0]*f_traintoscale.shape[1], 1])
scaler_X = MinMaxScaler(feature_range=(-1, 1))
scaler_X.fit(X_data_flatten)

y_data_flatten = np.reshape(y_traintoscale, [y_traintoscale.shape[0]*y_traintoscale.shape[1], y_traintoscale.shape[2]])
scaler_y = MinMaxScaler(feature_range=(-1, 1))
scaler_y.fit(y_data_flatten)

adam = Adam(learning_rate=0.001, decay=0.0001)
model = Sequential()
model.add(SimpleRNN(50, return_sequences=True, stateful=False, input_shape=(None, 1)))
model.add(Dense(1))
# model.summary()

model.compile(loss='mean_squared_error', optimizer=adam)   

string = './model/RNN_wts'
model.load_weights(string)

ft = [20, 25, 50, 75, 100]
for ftnum in range(0,5):
    print(ftnum)
    string = 'BW_SDOF_testing_data_FT_'+str(ft[ftnum])+'.mat'
    test_data = spi.loadmat(string)
    
    f = test_data['force'].T
    y = test_data['y1'].T
    
    print(f.shape, y.shape)

    with tf.device('/device:cpu:0'):
        X_data_flatten = np.reshape(f, [f.shape[0]*f.shape[1], 1])
        X_data_flatten_map = scaler_X.transform(X_data_flatten)
        X_test = np.reshape(X_data_flatten_map, [f.shape[0], f.shape[1], 1])

        predictions = model.predict(X_test)

        pred_disp = np.zeros([y.shape[0], y.shape[1], 1])
        for i in range(0,10000):
            pred_disp[i,:,:] = scaler_y.inverse_transform(predictions[i,:,:])

        plt.figure(figsize=[10,5])
        plt.plot(t,np.mean(pred_disp, axis = 0), 'b')
        plt.plot(t,np.mean(y, axis = 0), 'r--')
        plt.show()

        plt.figure(figsize=[10,5])
        plt.plot(t,np.var(pred_disp, axis = 0), 'b')
        plt.plot(t,np.var(y, axis = 0), 'r--')
        plt.show()

        # string = 'RNN3L_SDOF_BW_training_samples_'+str(150)+'_FT_'+str(ft[ftnum])+'.mat' 
        # spi.savemat(string,{'actual_y': y, 'predicted_y': np.squeeze(pred_disp), 'model_num': 150, 'fourier_terms': ft[ftnum]})
